In [14]:
import numpy as np
from search import astar_search
from PacProblem import PacProblem

## A* Search Solution

As an informed search algorithm, A\* takes into account information about the path cost together with an heuristic to evaluate which is the most promising path to take when it enters a state. For this evaluation, A\* chooses in state $n$ to proceed to the neighbor that gives the lowest $f(n) = g(n) + h(n)$, $g(n)$ being the exact path cost from starting state to $n$ and $h(n)$ the heuristic estimated cost from $n$ to goal state.

### Heuristic

How fast the agent reaches the goal in A\* depends highly on the heuristic implemented and how it affects the nodes expansion. In a problem like ours, where the maze configuration needs to be known in state $n$ so $f(n)$ can be calculated, it's specially important to pick a good heuristic - after all, the search space is exponentially large, as each subset of eaten food represents a different node, even with the agent in a fixed position.

A common approach to simpler pathfinding problems is to use the **Manhattan distance** as a heuristic, which is the distance between the agent and the goal positions measured along axes at right angles (i.e., $|x_1 - x_2| + |y_1 - y_2|$, given that the agent is in $(x_1, y_1)$ and the goal is to reach $(x_2, y_2)$). In our problem, however, is not realistic to estimate a good cost to goal using this distance, as the foods over the maze can make it very far from optimal.

Considering that, we implement the sum of Manhattan distances between the agent and all the foods as a heuristic, as it's highly possible that they will all be eaten in the optimal path. Notice that this sum can overestimate the optimal, because it's not true that all the foods will be eaten in the best path. As a overestimating heuristic, it breaks admissibility - that is, A\* is not guaranteed to find the optimal path. Even so, as our problem gives a high score to Pac-Man when it eats, we chose it expecting A\* will find good, if not optimal, paths in reasonable running times. 


In [17]:
def astar_heuristic(node):
    '''  '''